In [ ]:
## For Everyone and Anyone !! ** Do up-vote if you liked it **

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from dateutil import parser
import string
import re
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from spacy.lang.en.stop_words import STOP_WORDS

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gensim
from sklearn.preprocessing import StandardScaler,LabelEncoder
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
import scipy.stats as stats
import pylab 
import matplotlib.pyplot as plt

import en_core_web_lg
nlp = en_core_web_lg.load()

from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

print('Training shape : {}'.format(train.shape))
print('Testing shape : {}'.format(test.shape))

In [ ]:
train.head()

In [ ]:
## Feature Engg Prior to Cleaning :  to a generate some relevant features and then clenaing the irrelevnt data

In [ ]:
## Feature Engg !!

print('* ------------------------- for train data-------------------------------- *')

# the relevance of these feature will be tested on the later stages !!

train['word_count'] = train["excerpt"].apply(lambda x: len(str(x).split(" ")))
train['char_count'] = train["excerpt"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
train["num_unique_words"] = train["excerpt"].apply(lambda x: len(set(str(x).split())))
train["num_stopwords"] = train["excerpt"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
train["num_punctuations"] =train['excerpt'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
train["num_words_upper"] = train["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
train["num_words_title"] = train["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
train["mean_word_len"] = train["excerpt"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

print('* ------------------------- for test data-------------------------------- *')

test['word_count'] = test["excerpt"].apply(lambda x: len(str(x).split(" ")))
test['char_count'] = test["excerpt"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
test["num_unique_words"] = test["excerpt"].apply(lambda x: len(set(str(x).split())))
test["num_stopwords"] = test["excerpt"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_punctuations"] =test['excerpt'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test["num_words_upper"] = test["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_title"] = test["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["mean_word_len"] = test["excerpt"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train.head(2)

In [ ]:
## Cleaning !!

def is_valid_date(date_str): # sub function
    try:
        parser.parse(date_str)
        return True
    except:
#         print(date_str)
        return False
    
def date_removal(data):
    new_list = [' '.join([w for w in line.split() if not is_valid_date(w)]) for line in data]
    return (new_list[0])

def stemmer_and_stopWord(doc):
    
    doc= nlp(doc)
    token_list = []
    for token in doc:
        lemma = token.lemma_
        if lemma == '-PRON-' or lemma == 'be':
            lemma = token.text
        token_list.append(lemma)

    stemmed = token_list
    
    # Create list of word tokens after removing stopwords
    
    filtered_sentence =[] 
    for word in stemmed[:100]:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    return (' '.join(filtered_sentence))

def normaliz(filtered_sentence):
        
    words = [str(word).lower() for word in filtered_sentence.split()]
    return  ' '.join(words[:100])

# For number removal ~

def numbers_removal(data):
    s = [data]
    result = ''.join([i for i in s if not i.isdigit()])
    return (result)
# For punchuation & double white spaces ~

def punch_removal(words):
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in [words]]
    return re.sub(' +', ' ', stripped[:100][0])


def cleaner(data):
    string = [data]
    string = date_removal(string)
    string = numbers_removal(string)
    string = punch_removal(string)
    string = stemmer_and_stopWord(string)
    string = normaliz(string)
    return string

print('* --------------- for train data --------------- *')

list=[]
for i in train['excerpt']: # cleansing !!
    list.append(cleaner(i))

train['excerpt']=pd.Series(list) # updating the attributee !!

print('* --------------- for test data --------------- *')

list=[]
for i in test['excerpt']: # cleansing !!
    list.append(cleaner(i))

test['excerpt']=pd.Series(list) # updating the attributee !!



In [ ]:
train.head()

In [ ]:
## Processing on Text data (Excerpt) : Starts !!

In [ ]:
# list ofString in Title
titles_list = [title for title in train['excerpt']]
# Collapse the list of strings into a single long string for processing
big_title_string = ' '.join(titles_list)
# converting string into words
tokens = word_tokenize(big_title_string)
# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]
# Filter out stopwords, exampe At, for, in , is for getting relevant words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [word for word in words if not word in stop_words]
# Print first 10 words
words[:10]

In [ ]:
# Load Word2Vec model (trained on an enormous Google corpus)
model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectors/GoogleNews-vectors-negative300.bin', binary = True) 
# Check dimension of word vectors
model.vector_size

In [ ]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.index_to_key]
# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.index_to_key]
# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)
# Cast to a dict so we can turn it into a dataframe
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.shape

In [ ]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.index_to_key]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors
# Here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.index_to_key for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes
    a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [ ]:
# Preprocess the corpus
corpus = [preprocess(title) for title in titles_list]
# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))
# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))

In [ ]:
# Initialize an array for the size of the corpus
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
X = np.array(x) # list to array

In [ ]:
if test.shape[0] > 30:
    component = 30
else:
    component = test.shape[0]

print(test.shape[0], component)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=component, random_state=10) 
# as a reminder, x is the array with our 300-dimensional vectors
reduced_vecs = pca.fit_transform(x) 
df_w_vectors = pd.DataFrame(reduced_vecs) 
df_w_vectors ['excerpt'] = titles_list 
# Use pd.concat to match original titles with their vectors
main_w_vectors = pd.concat ((df_w_vectors, train), axis = 1)
# Get rid of vectors that could not be matched with the main_df
main_w_vectors .dropna(axis=0, inplace=True)

In [ ]:
# Drop all non-numeric, non-dummy columns, for feeding into the models
cols_to_drop = ['url_legal', 'license', 'excerpt', 'id']
data = main_w_vectors.drop(columns = cols_to_drop) 

In [ ]:
print(data.columns)
data.head(2)

In [ ]:
## ----------- Modeling ------------ ##

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet,SGDRegressor
from sklearn.model_selection import GridSearchCV

X=data.drop(['target', 'standard_error'], axis=1)
Y=data['target']


fre = RandomForestRegressor(max_depth=8,n_estimators=10,n_jobs=2)
fre.fit(X,Y.values.ravel())
print(fre.feature_importances_) 
feat_importances = pd.Series(fre.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dropout, GRU, Activation, Embedding, Bidirectional,SpatialDropout1D, BatchNormalization, Conv1D, MaxPool1D

In [ ]:
print(X.shape,Y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_train=X_train.reshape(X_train.shape[0], X_train.shape[1],1) # it needs to be a nummpy array
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1],1)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# convolution !!

epochs=10
cat_model=Sequential()

cat_model.add(Conv1D(filters=32, activation='relu', kernel_size=2, input_shape=(X_test.shape[1],1)))
cat_model.add(Conv1D(filters=64, activation='relu', kernel_size=2))
cat_model.add(Dropout(0.2))
cat_model.add(Flatten())
cat_model.add(Dense(128, activation='relu'))
cat_model.add(Dropout(0.2))
cat_model.add(Dense(1,activation='linear'))

In [ ]:
cat_model.summary()

cat_model.compile( optimizer=Adam(lr=0.05), loss='mae', metrics=['mae'])
history=cat_model.fit(X_train,y_train, epochs=epochs, validation_data=(X_test,y_test), verbose=2)

In [ ]:
accr = cat_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
#--------------------------------------------------------------#

In [ ]:
## test data processing !!

In [ ]:
# list ofString in Title
titles_list = [title for title in test['excerpt']]
# Collapse the list of strings into a single long string for processing
big_title_string = ' '.join(titles_list)
from nltk.tokenize import word_tokenize
# converting string into words
tokens = word_tokenize(big_title_string)
# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]
# Filter out stopwords, exampe At, for, in , is for getting relevant words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [word for word in words if not word in stop_words]

# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.index_to_key]
# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.index_to_key]
# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)
# Cast to a dict so we can turn it into a dataframe
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.shape

In [ ]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.index_to_key]
# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.index_to_key]
# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)
# Cast to a dict so we can turn it into a dataframe
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.shape

In [ ]:
# Preprocess the corpus
corpus = [preprocess(title) for title in titles_list]
# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))
# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))

# Initialize an array for the size of the corpus
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
X = np.array(x) # list to array

In [ ]:
pca = PCA(n_components=component, random_state=10) 
reduced_vecs = pca.fit_transform(x) 
df_w_vectors = pd.DataFrame(reduced_vecs) 
df_w_vectors [ 'excerpt'] = titles_list 
main_w_vectors = pd.concat ((df_w_vectors, test), axis = 1)

# Drop all non-numeric, non-dummy columns, for feeding into the models
cols_to_drop = ['url_legal', 'license', 'excerpt']

data_only_df=pd.DataFrame()
data_only_df = main_w_vectors.drop(columns = cols_to_drop)

In [ ]:
data_only_df.columns

In [ ]:
title=data_only_df.drop(['id'], axis=1)
title = scaler.fit_transform(title)

title=title.reshape(title.shape[0], title.shape[1],1) # it needs to be a nummpy array
# headline=data_only_df['target']
submission=pd.DataFrame()

In [ ]:
y_predtitle=cat_model.predict(title)
y_predtitle

In [ ]:
submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission['target'] = y_predtitle
submission

In [ ]:
submission.to_csv("submission.csv",index=False)